In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

In [2]:
df = pd.read_excel('../data/raw/farm_site_boundary.xlsx')

In [3]:
def string_to_polygon(s):
    # Replace single quotes with double quotes for valid JSON format
    s = s.replace("'", '"')
    # Turn the string into a list of point dictionaries
    points_list = eval(s)
    # Create a Polygon object from the list of point dictionaries
    polygon = Polygon([(point['lng'], point['lat']) for point in points_list])
    return polygon

In [4]:
df['polygon_geometry'] = df['location_grid_points'].apply(string_to_polygon)
df['point_geometry'] = [Point(x, y) for x, y in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry='point_geometry')

In [5]:
gdf.crs = "EPSG:4326"
gdf.head(2)

,farm_id,country_name,latitude,longitude,location_id,type,location_area_m2,location_grid_points,polygon_geometry,point_geometry
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,Canada,49.059192,-122.882005,605c6008-ab7e-11eb-8a94-0242ac130002,farm_site_boundary,211908.0,"[{""lat"": 49.05928816695972, ""lng"": -122.883487...",POLYGON ((-122.88348775768395 49.0592881669597...,POINT (-122.88200 49.05919)
1,36322d78-b2ac-11eb-9edf-0242ac130002,Canada,49.036135,-122.713490,931eeb88-b2ad-11eb-b6ac-0242ac130002,farm_site_boundary,21388.0,"[{""lat"": 49.0361106915654, ""lng"": -122.7183954...",POLYGON ((-122.71839542729994 49.0361106915654...,POINT (-122.71349 49.03613)


In [6]:
provinces = gpd.read_file("../data/external/province_100m")
provinces = provinces.to_crs("EPSG:4326") 
provinces.head(3)

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
0,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,Newfoundland and Labrador,Terre-Neuve-et-Labrador,N.L.,T.-N.-L.,"MULTIPOLYGON (((-57.40256 54.14965, -57.38429 ..."
1,11,Prince Edward Island / Île-du-Prince-Édouard,Prince Edward Island,Île-du-Prince-Édouard,P.E.I.,Î.-P.-É.,"MULTIPOLYGON (((-61.98300 46.45775, -61.97204 ..."
2,12,Nova Scotia / Nouvelle-Écosse,Nova Scotia,Nouvelle-Écosse,N.S.,N.-É.,"MULTIPOLYGON (((-61.90233 45.87878, -61.89449 ..."


In [7]:
farm_prov = gpd.sjoin(gdf,
                          provinces[["PRENAME", "geometry"]], 
                          how='left', 
                          predicate='intersects'
                          ).drop(columns=['index_right'])
farm_prov.head(3)

,farm_id,country_name,latitude,longitude,location_id,type,location_area_m2,location_grid_points,polygon_geometry,point_geometry,PRENAME
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,Canada,49.059192,-122.882005,605c6008-ab7e-11eb-8a94-0242ac130002,farm_site_boundary,211908.0,"[{""lat"": 49.05928816695972, ""lng"": -122.883487...",POLYGON ((-122.88348775768395 49.0592881669597...,POINT (-122.88200 49.05919),British Columbia
1,36322d78-b2ac-11eb-9edf-0242ac130002,Canada,49.036135,-122.713490,931eeb88-b2ad-11eb-b6ac-0242ac130002,farm_site_boundary,21388.0,"[{""lat"": 49.0361106915654, ""lng"": -122.7183954...",POLYGON ((-122.71839542729994 49.0361106915654...,POINT (-122.71349 49.03613),British Columbia
2,dee27326-b2af-11eb-a871-0242ac130002,Canada,43.944641,-79.730420,607eb4a8-b2b0-11eb-b10e-0242ac130002,farm_site_boundary,211695.0,"[{""lat"": 43.94351188354036, ""lng"": -79.7265726...",POLYGON ((-79.72657263766844 43.94351188354036...,POINT (-79.73042 43.94464),Ontario


In [8]:
farm_prov.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 123 entries, 0 to 122
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   farm_id               123 non-null    object  
 1   country_name          123 non-null    object  
 2   latitude              123 non-null    float64 
 3   longitude             123 non-null    float64 
 4   location_id           123 non-null    object  
 5   type                  123 non-null    object  
 6   location_area_m2      123 non-null    float64 
 7   location_grid_points  123 non-null    object  
 8   polygon_geometry      123 non-null    object  
 9   point_geometry        123 non-null    geometry
 10  PRENAME               123 non-null    object  
dtypes: float64(3), geometry(1), object(7)
memory usage: 11.5+ KB


In [9]:
ecodistrict = gpd.read_file("../data/external/slc_dissolved_ecodistrict")
ecodistrict = ecodistrict.to_crs('EPSG:4326')
ecodistrict.head(3)

,ECO_ID,AREA,PERIMETER,geometry
0,1,1.379902,18.141771,"POLYGON ((-81.82100 80.84613, -81.86701 80.842..."
1,2,7.099515,65.471834,"POLYGON ((-78.14311 79.33501, -78.24853 79.337..."
2,3,2.019555,17.852421,"POLYGON ((-91.08031 79.05304, -91.07503 79.030..."


In [10]:
farm_prov = farm_prov.to_crs("EPSG:4326") 
farm_prov.set_geometry('point_geometry', inplace=True)

farm_ecod_prov = gpd.sjoin(farm_prov, 
                          ecodistrict[["ECO_ID", "geometry"]],
                          how='left', 
                          predicate='within'
                         ).drop(columns=['index_right'])
farm_ecod_prov.head(3)

,farm_id,country_name,latitude,longitude,location_id,type,location_area_m2,location_grid_points,polygon_geometry,point_geometry,PRENAME,ECO_ID
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,Canada,49.059192,-122.882005,605c6008-ab7e-11eb-8a94-0242ac130002,farm_site_boundary,211908.0,"[{""lat"": 49.05928816695972, ""lng"": -122.883487...",POLYGON ((-122.88348775768395 49.0592881669597...,POINT (-122.88200 49.05919),British Columbia,959
1,36322d78-b2ac-11eb-9edf-0242ac130002,Canada,49.036135,-122.713490,931eeb88-b2ad-11eb-b6ac-0242ac130002,farm_site_boundary,21388.0,"[{""lat"": 49.0361106915654, ""lng"": -122.7183954...",POLYGON ((-122.71839542729994 49.0361106915654...,POINT (-122.71349 49.03613),British Columbia,959
2,dee27326-b2af-11eb-a871-0242ac130002,Canada,43.944641,-79.730420,607eb4a8-b2b0-11eb-b10e-0242ac130002,farm_site_boundary,211695.0,"[{""lat"": 43.94351188354036, ""lng"": -79.7265726...",POLYGON ((-79.72657263766844 43.94351188354036...,POINT (-79.73042 43.94464),Ontario,562


In [11]:
climate = pd.read_csv("../data/raw/Holos/ecodistrict_to_ecozone_mapping.csv")
climate.head(3)

,Ecodistrict,Ecozone,Province,PMayToOct,PEMayToOct,Ftopo,SoilType,SoilTexture
0,358,Boreal Shield West,Manitoba,348,453,0.00,Brown Chernozem,Coarse
1,358,Boreal Shield West,Saskatchewan,348,453,0.00,Brown Chernozem,Coarse
2,371,Boreal Shield West,Manitoba,376,581,0.33,Black/Gray Chernozem,Fine


In [12]:
farm_ecoid_climate = pd.merge(farm_ecod_prov, climate,  
                              how='left', 
                              left_on=['ECO_ID','PRENAME'], 
                              right_on = ['Ecodistrict','Province'])

In [13]:
farm_ecoid_climate.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   farm_id               123 non-null    object  
 1   country_name          123 non-null    object  
 2   latitude              123 non-null    float64 
 3   longitude             123 non-null    float64 
 4   location_id           123 non-null    object  
 5   type                  123 non-null    object  
 6   location_area_m2      123 non-null    float64 
 7   location_grid_points  123 non-null    object  
 8   polygon_geometry      123 non-null    object  
 9   point_geometry        123 non-null    geometry
 10  PRENAME               123 non-null    object  
 11  ECO_ID                123 non-null    int64   
 12  Ecodistrict           123 non-null    int64   
 13  Ecozone               123 non-null    object  
 14  Province              123 non-null    object  
 15

In [14]:
farm_ecoid_climate.head(3)

,farm_id,country_name,latitude,longitude,location_id,type,location_area_m2,location_grid_points,polygon_geometry,point_geometry,PRENAME,ECO_ID,Ecodistrict,Ecozone,Province,PMayToOct,PEMayToOct,Ftopo,SoilType,SoilTexture
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,Canada,49.059192,-122.882005,605c6008-ab7e-11eb-8a94-0242ac130002,farm_site_boundary,211908.0,"[{""lat"": 49.05928816695972, ""lng"": -122.883487...",POLYGON ((-122.88348775768395 49.0592881669597...,POINT (-122.88200 49.05919),British Columbia,959,959,Pacific Maritime,British Columbia,480,506,6.53,Brown Chernozem,Medium
1,36322d78-b2ac-11eb-9edf-0242ac130002,Canada,49.036135,-122.713490,931eeb88-b2ad-11eb-b6ac-0242ac130002,farm_site_boundary,21388.0,"[{""lat"": 49.0361106915654, ""lng"": -122.7183954...",POLYGON ((-122.71839542729994 49.0361106915654...,POINT (-122.71349 49.03613),British Columbia,959,959,Pacific Maritime,British Columbia,480,506,6.53,Brown Chernozem,Medium
2,dee27326-b2af-11eb-a871-0242ac130002,Canada,43.944641,-79.730420,607eb4a8-b2b0-11eb-b10e-0242ac130002,farm_site_boundary,211695.0,"[{""lat"": 43.94351188354036, ""lng"": -79.7265726...",POLYGON ((-79.72657263766844 43.94351188354036...,POINT (-79.73042 43.94464),Ontario,562,562,Mixedwood Plains,Ontario,461,586,2.03,Eastern Canada,Medium


In [15]:
farm_df = farm_ecoid_climate[["farm_id", "latitude", "longitude"]]
farm_df

,farm_id,latitude,longitude
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005
1,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490
2,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420
3,1e3b98f8-ca44-11eb-8062-0242ac130002,48.995126,-58.045602
4,33851eae-dad5-11eb-9c2f-0242ac130002,46.636528,-71.042723
...,...,...,...
118,3531b218-e6c0-11ee-a6cc-0242ac150002,43.717398,-79.625701
119,2a03cf4c-e9e6-11ee-8667-0242ac150002,49.199923,-119.762455
120,4316e7b6-eb90-11ee-807f-0242ac150002,51.272857,-120.173324
121,4316e7b6-eb90-11ee-807f-0242ac150002,51.272857,-120.173324


In [16]:
# The following yield is from Google, US data. kg/m^2
crop_yield = {
    'Soybean': 0.27, 
    'Potato': 3.58,
    'Wheat': 0.28
}

In [17]:
AREA = 10000
START_YEAR = 2021
END_YEAR = 2021

In [18]:
replicated_df = farm_df.loc[farm_df.index.repeat(3)].reset_index(drop=True)
replicated_df.head(9)

,farm_id,latitude,longitude
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005
1,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005
2,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005
3,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490
4,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490
5,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490
6,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420
7,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420
8,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420


In [19]:
replicated_df['common_crop_name'] = list(crop_yield.keys()) * (len(replicated_df) // len(crop_yield))
replicated_df['yield_kg_per_m2'] = list(crop_yield.values()) * (len(replicated_df) // len(crop_yield))

replicated_df.head(9)

,farm_id,latitude,longitude,common_crop_name,yield_kg_per_m2
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Soybean,0.27
1,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Potato,3.58
2,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Wheat,0.28
3,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Soybean,0.27
4,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Potato,3.58
5,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Wheat,0.28
6,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Soybean,0.27
7,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Potato,3.58
8,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Wheat,0.28


In [20]:
replicated_df['area_in_m2'] = AREA
replicated_df['start_year'] = 2021
replicated_df['end_year'] = 2021

In [21]:
replicated_df.head(9)

,farm_id,latitude,longitude,common_crop_name,yield_kg_per_m2,area_in_m2,start_year,end_year
0,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Soybean,0.27,10000,2021,2021
1,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Potato,3.58,10000,2021,2021
2,7a2e6fcc-ab78-11eb-a4ad-0242ac130002,49.059192,-122.882005,Wheat,0.28,10000,2021,2021
3,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Soybean,0.27,10000,2021,2021
4,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Potato,3.58,10000,2021,2021
5,36322d78-b2ac-11eb-9edf-0242ac130002,49.036135,-122.713490,Wheat,0.28,10000,2021,2021
6,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Soybean,0.27,10000,2021,2021
7,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Potato,3.58,10000,2021,2021
8,dee27326-b2af-11eb-a871-0242ac130002,43.944641,-79.730420,Wheat,0.28,10000,2021,2021


In [22]:
replicated_df.to_csv('../data/test/LiteFarm_CA_HypotheticalFarmCropYields.csv', index=False)